<a href="https://colab.research.google.com/github/g185/AMLrepository/blob/main/RecipeNet/AML_Project_Giuliano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Download and unzip Dataset***



*   Create folders
*   Download Recipe 5K + Annotations in drive
*   Unzip files in folder



In [6]:
#Create folders for models and datasets
!mkdir "/content/drive/My Drive/RecipeNet" 
!mkdir "/content/drive/My Drive/RecipeNet/datasets" 
!mkdir "/content/drive/My Drive/RecipeNet/datasets/download" 
!mkdir "/content/drive/My Drive/RecipeNet/datasets/extracted" 
!mkdir "/content/drive/My Drive/RecipeNet/model" 

#Scarica dataset
#Trascina i 3 zip

#Unzip
#!unzip "/content/drive/My Drive/RecipeNet/datasets/download/Recipes5k.zip" -d "/content/drive/My Drive/RecipeNet/datasets/extracted"
!unzip "/content/drive/My Drive/RecipeNet/datasets/download/archive.zip" -d "/content/drive/My Drive/RecipeNet/datasets/extracted"
#!unzip "/content/drive/My Drive/RecipeNet/datasets/download/Ingredients101.zip" -d "/content/drive/My Drive/RecipeNet/datasets/extracted"




mkdir: cannot create directory ‘/content/drive/My Drive/RecipeNet’: File exists
mkdir: cannot create directory ‘/content/drive/My Drive/RecipeNet/datasets’: File exists
mkdir: cannot create directory ‘/content/drive/My Drive/RecipeNet/datasets/download’: File exists
mkdir: cannot create directory ‘/content/drive/My Drive/RecipeNet/datasets/extracted’: File exists
mkdir: cannot create directory ‘/content/drive/My Drive/RecipeNet/model’: File exists
Archive:  /content/drive/My Drive/RecipeNet/datasets/download/archive.zip
replace /content/drive/My Drive/RecipeNet/datasets/extracted/food_c101_n1000_r384x384x3.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


#***Imports and Drive Mount***

In [1]:
#Imports
import sys
import time
import os
import json
import numpy as np 
import pandas as pd 
from tqdm import tqdm, tqdm_notebook
import sys
import cv2
import torch
import torch.utils.data as data
from torch.autograd import Variable
import torch.nn as nn
from torchvision import datasets, transforms
from glob import glob
import matplotlib.pyplot as plt
import torchvision.models as models

#Cuda
device = torch.device("cuda")
torch.manual_seed(42) # try and make the results more reproducible
print(torch.cuda.get_device_name(0))

#Drive Mount
from google.colab import drive
drive.mount('/content/drive')


cuda
Mounted at /content/drive
Tesla P100-PCIE-16GB


#***Parameters***


In [15]:
params = {}
params["root"] = "/content/drive/My Drive/RecipeNet/"
params["images_dir"] = params["root"] + "datasets/extracted/images" 
params["ingredients_per_class"] = params["root"] + "datasets/extracted/Ingredients101/Annotations/ingredients_simplified.txt" 
params["classes"] = "/content/drive/My Drive/RecipeNet/datasets/extracted/Ingredients101/Annotations/classes.txt" 
params["baseIngredients_dir"] = "/content/drive/My Drive/RecipeNet/datasets/extracted/Ingredients101/ingredients_simplification/baseIngredients.txt" 
params["epochs"] = 35
batch_size = 64
params["img_size"] = (384,384)
params["fast_training"] = False

#***Data extraction and preprocessing***


In [36]:
f = open(params["ingredients_per_class"], "r")
ingredients = f.read().split('\n')
f.close()

f = open(params["classes"], "r")
classes = f.read().split('\n')
f.close()

f = open(params["baseIngredients_dir"], "r")
base_ing = f.read().split('\n')
base_ing = base_ing[0].split(",")
f.close()

#train
f = open(params["root"] + "datasets/extracted/Ingredients101/Annotations/train_images.txt", "r")
train_images = f.read().split('\n')
f.close()
f = open(params["root"] + "datasets/extracted/Ingredients101/Annotations/train_labels.txt", "r")
train_labels = f.read().split('\n')
f.close()

#val
f = open(params["root"] + "datasets/extracted/Ingredients101/Annotations/val_images.txt", "r")
val_images = f.read().split('\n')
f.close()
f = open(params["root"] + "datasets/extracted/Ingredients101/Annotations/val_labels.txt", "r")
val_labels = f.read().split('\n')
f.close()

#test
f = open(params["root"] + "datasets/extracted/Ingredients101/Annotations/test_images.txt", "r")
test_images = f.read().split('\n')
f.close()
f = open(params["root"] + "datasets/extracted/Ingredients101/Annotations/test_labels.txt", "r")
test_labels = f.read().split('\n')
f.close()


new_ingredients = []
for arr in ingredients:
    arr = arr.split(",")
    new_ingredients.append(arr)


from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

df = pd.DataFrame(mlb.fit_transform(new_ingredients),columns=mlb.classes_) #binary encode ingredients


df["target"] = classes
food_dict = df



train_images = ["../input/food41/images/" + s + ".jpg" for s in train_images]
all_img_df = pd.DataFrame({'path': train_images, 'class_id': train_labels})
val_images = ["../input/food41/images/" + s + ".jpg" for s in val_images]
val_img_df = pd.DataFrame({'path': val_images, 'class_id': val_labels})
test_images = ["../input/food41/images/" + s + ".jpg" for s in test_images]
test_img_df = pd.DataFrame({'path': test_images, 'class_id': test_labels})



all_img_df = all_img_df[:-1]
val_img_df = val_img_df[:-1]
test_img_df = test_img_df[:-1]


food_dict = food_dict.drop('', 1)

food_dict
"""
#Dataframe for train images
new_data = []
for index, row in all_img_df.iterrows():
    #get binary encoding ingredients from lookup
    food = row["class_name"]
    path = row["path"]
    class_id = row["class_id"]
    binary_encod = food_dict.loc[food_dict["target"] == food]
    binary_encod["path"] = path
    binary_encod["class_id"] = class_id
    #print(binary_encod)
    #print((list(binary_encod.columns.values)))
    #print(len(np.array(binary_encod)[0]))
    new_data.append(np.array(binary_encod)[0])


    
col_names = list(binary_encod.columns.values)
train_df = pd.DataFrame(new_data, columns = col_names)

val_data = []
for index, row in val_img_df.iterrows():
    #get binary encoding ingredients from lookup
    food = row["class_name"]
    path = row["path"]
    class_id = row["class_id"]
    binary_encod = food_dict.loc[food_dict["target"] == food]
    binary_encod["path"] = path
    binary_encod["class_id"] = int(class_id)
    val_data.append(np.array(binary_encod)[0])

    
val_df = pd.DataFrame(val_data, columns = col_names)
"""

,adobo,almond,anaheim chile,anchovy,anise,apple,arborio rice,asiago,avocado,baby arugula,baby back ribs,baby bok choy,bacon,baguette,baking,baklava,balsamic vinegar,basil,bay,bay leaf,beans,beef,beer,beets,berries,bird chile,biscuits,black pepper,bran,bread,brewed espresso,brie,broth,brown rice,brown sugar,buns,butter,cabbage,cajun,cake,...,seeds,serrano chilies,shallot,shiitake,shortening,shoulder roast,shrimp,sirloin,snails,soda,soy,soybean sprouts,spaghetti,spinach,spray,spring roll wrappers,steak,strawberries,sugar,sushi grade tuna,tahini,tartar,thyme,tofu,tomato,tortillas,turmeric,unflavored gelatin,vanilla,vanilla essence,vermouth,vinegar,water,wheat tortillas,whipping,white wine,worcestershire,yeast,zucchini,target
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,apple_pie
1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,baby_back_ribs
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,baklava
3,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,beef_carpaccio
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,beef_tartare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,takoyaki
98,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,tiramisu
99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,tuna_tartare
100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,waffles


#***Model definition***